Otras herramientas
==========================================

## Spark SQL

-   Interfaz para trabajar con datos estructurados y semiestructurados

-   Capacidades principales

    -   Lee datos de una gran variedad de fuentes: JSON, Hive, Parquet…

    -   Permite consultas SQL, tanto desde programas Spark como externas
        usando conectores estándar (JDBC/ODBC)

    -   Integra SQL y código Spark normal (en Python/Java/Scala)

### Conceptos básicos de Spark SQL

-   Contexto: punto de entrada

    -   `SQLContext`: Contexto básico

    -   `HiveContext`: Contexto basado en Hive

-   `DataFrame`: colección distribuida de datos organizada en columnas
    con nombre

    -   Colección distribuida de datos organizacos en columnas con nombre

    -   Conceptualmente equivalente a una tabla en una BD o a un
        dataframe en R o Pandas

-   `DataSet`: nuevo tipo de datos añadido en Spark 1.6
    - Intenta proporcionar los beneficioos de los RDDs con las optimizaciones que proporciona el motor de ejecución de Spark SQL
    - De momento, sólo disponible en Scala y Java
    - En [Java](http://spark.apache.org/docs/latest/api/java/index.html "Interface Row") y [Scala](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Row "trait Row extends Serializable"), un DataFrame es un DataSet de Rows

### Spark SQL: ejemplo fichero JSON

In [1]:
from __future__ import print_function
from test_helper import Test

from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)
df = sqlContext.read.json("datos/gente.json")

print("Tabla completa")
df.show()

print("Estadísticas")
df.describe("edad").show()

print("Esquema inferido")
df.printSchema()

print("Columna nombre")
df.select("nombre").show()

print("Filtra por edad")
df.filter(df["edad"] > 21).show()

print("Agrupa por edad")
df.groupBy("edad").count().show()

print("Consulta SQL")
# Registra la tabla para usar SQL
df.registerTempTable("gente")
teens = sqlContext.sql("SELECT nombre FROM gente WHERE edad >= 13 AND edad <= 19")
teens.show()

ImportError: No module named test_helper

In [ ]:
lines = sc.textFile("datos/gente.txt")
p = lines.map(lambda l: l.split(","))
ne = p.map(lambda p: Row(nombre=p[0], edad=int(p[1])))
df = sqlContext.createDataFrame(ne)                              

print("Tabla completa")
df.show()

print("Consulta SQL")
# Registra la tabla para usar SQL
df.registerTempTable("gente2")
teens2 = sqlContext.sql("SELECT nombre FROM gente2 WHERE edad >= 13 AND edad <= 19")
teens2.show()

# Añade la tabla a la cache
sqlContext.cacheTable("gente2")

### Spark SQL: UDFs

-   Funciones definidas por el usuario

In [ ]:
from pyspark.sql.types import IntegerType

# Registra una función que devuelve 
# un entero con la longitud del string
sqlContext.registerFunction("lon", lambda x: len(x), IntegerType())

# Aplica la función a la columna nombre                             
lndf = sqlContext.sql("SELECT lon(nombre) FROM gente2")
print("Número de caracteres de los nombres")
lndf.show()

## Spark Streaming

-   Procesamiento escalable, *high-throughput* y tolerante a fallos de
    flujos de datos

<img src="figs/streaming-flow.png" alt="Flujo de Spark Streaming" style="width: 800px;"/>

-   Entrada desde muchas fuentes: Kafka, Flume, Twitter, ZeroMQ, Kinesis
    o sockets TCP

### Arquitectura de Spark Streaming

Abstracción principal: DStream (*discretized stream*).

-   Representa un flujo continuo de datos

![image](figs/dstreams.png)

Arquitectura *micro-batch*

-   Los datos recibidos se agrupan en batches

-   Los batches se crean a intervalos regulares (batch interval)

-   Cada batch forma un RDD, que es procesado por SPARK

-   Adicionalmente: transformaciones con estado mediante

    -   Operaciones con ventanas

    -   Tracking del estado por cada clave

### Spark Streaming: WordCount en red
```python
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Contexto con dos threads locales
sc = SparkContext("local[2]", "NetworkWordCount")
sc.setLogLevel("WARN")
# Contexto Streaming con un batch interval de 1 s
ssc = StreamingContext(sc, 5)

# DStream que conecta a localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

# Ejecuta un WordCount
counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .reduceByKey(lambda a, b: a+b)
counts.pprint()

ssc.start() # Inicia la computacion
ssc.awaitTermination() # Espera a que termine
```
Para ejecutar el ejemplo:

   1. En un terminal usa netcat como un servidor en el puerto 9999

    `$ nc -lk 9999`

   2. En otro terminal, inicia el script Spark

    `$ spark-submit netwc.py localhost 9999`

   3. Escribe líneas en el terminal del netcat, que serán recogidas y procesadas por el script



### Spark Streaming: WordCount con estado
```python
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
sc = SparkContext("local[2]", "StateWordCount")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("cpdir") # Activa checkpoint

def updateFunc(new_values, last_sum):
    return sum(new_values) + (last_sum or 0)

lines = ssc.socketTextStream("localhost", 9999)
counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .updateStateByKey(updateFunc)
counts.pprint()

ssc.start() # Inicia la computacion
ssc.awaitTermination() # Espera a que termine
```

## MLlib: Machine Learning Library

Librería de algoritmos paralelos de ML para datos masivos

-   Algoritmos de clasificación y regresión, clustering, filtrado
    colaborativo y recomendación, reducción de dimensionalidad, y
    primitivas de bajo nivel

-   API de alto nivel para ML pipelines

Dos paquetes:

-   spark.mllib: API original, basada en RDDs

-   spark.ml: API de alto nivel, basada en DataFrames

Documentación:
[spark.apache.org/docs/latest/mllib-guide.html](http://spark.apache.org/docs/latest/mllib-guide.html)
y
[spark.apache.org/docs/latest/ml-guide.html](http://spark.apache.org/docs/latest/ml-guide.html)

In [ ]:
# Usa KMeans para agrupar datos de vectores dispersos en dos clusters.
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector
import numpy as np

# Define un array de 4 vectores dispersos, de 3 elementos cada uno
sparse_data = [
     SparseVector(3, {1: 1.0}),
     SparseVector(3, {1: 1.1}),
     SparseVector(3, {2: 1.0}),
     SparseVector(3, {2: 1.1})
 ]

# Construye el modelo (agrupa los datos en 2 clusters)
model = KMeans.train(sc.parallelize(sparse_data), 2, initializationMode="k-means||",\
                                     seed=50, initializationSteps=5, epsilon=1e-4)

print("Centros de los clusters: {0}".format(model.clusterCenters))

Test.assertEquals(model.predict(sparse_data[0]),model.predict(sparse_data[1]))
Test.assertEquals(model.predict(sparse_data[2]),model.predict(sparse_data[3]))

# Salva el modelo en un directorio temporal
import os, tempfile
path = tempfile.mkdtemp()
model.save(sc, path)

# Vuelve a cargar el modelo
sameModel = KMeansModel.load(sc, path)
sameModel.predict(sparse_data[0]) == model.predict(sparse_data[0])
Test.assertEquals(sameModel.predict(sparse_data[0]),model.predict(sparse_data[0]))

# Borra el directorio temporal
from shutil import rmtree
try:
     rmtree(path)
except OSError:
     pass

## GraphX: procesamiento de grafos

Programación paralela de grafos con Spark

-   Principal abstracción:
    [*Graph*](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.graphx.Graph)

    -   Multigrafo dirigido con propiedades asignadas a vértices y
        aristas

    -   Extensión de los RDDs

-   Incluye constructores de grafos, operadores básicos (*reverse*,
    *subgraph*…) y algoritmos de grafos (*PageRank*, *Triangle
    Counting*…)
- Actualmente, no disponible en pySpark (solo Scala y Java)

Documentación:
[spark.apache.org/docs/latest/graphx-programming-guide.html](http://spark.apache.org/docs/latest/graphx-programming-guide.html)
